In [10]:
# Yuke Luo
from gurobipy import *
import mysql.connector
m = Model("Final model") #create model
m.remove(m.getVars())
m.remove(m.getConstrs())

In [11]:
db = mysql.connector.connect(user='root',password='root',database='final',auth_plugin='mysql_native_password')

In [12]:
cur = db.cursor(buffered=True)

In [13]:
dcquery = "SELECT dc_id, capacity FROM dc"
storequery = "SELECT store_id, requirements FROM store"
mileagequery = "SELECT dc_id,store_id,mileage FROM mileage"

In [14]:
cur.execute(dcquery)
records = cur.fetchall()
dcinfo = []
for row in records:
    dcinfo.append(row)

In [15]:
cur.execute(storequery)
records = cur.fetchall()
storeinfo = []
for row in records:
    storeinfo.append(row)

In [16]:
cur.execute(mileagequery)
records = cur.fetchall()
mileageinfo = []
for row in records:
    mileageinfo.append(row)

In [17]:
num_stores = range(1100)
num_dcs = range(10)
mileages =[]
for i in range(0,11000,1100):
    mileages.append(mileageinfo[i:i+1100])
store = []
for i in range(len(num_stores)):
    mile = []
    for j in range(len(num_dcs)): 
        if storeinfo[i][0] == mileages[j][i][1]:
            mile.append(mileages[j][i][2])
    store.append(mile)


# Decision Variables
binary=[[m.addVar(vtype=GRB.BINARY, name='binary'+str(i)+'_'+str(j)) for j in range(len(num_dcs))]for i in range(len(num_stores))]
demand= [ m.addVar(vtype=GRB.INTEGER, name= 'demand' + str(i)) for i in range(len(num_stores))]
m.update()

In [18]:
# Objective Funcion
fixed_price = 200
unit_price = 0.75
var_amount = 0
fixed_cost=fixed_price*sum(demand)
going = []
for i in range(len(num_stores)):
    go = []
    for j in range(len(num_dcs)):
        go.append(binary[i][j]*storeinfo[i][1])
    going.append(go)
for i in range(len(num_stores)):
    for j in range(len(num_dcs)):
        var_amount += store[i][j]*binary[i][j]*going[i][j]
m.setObjective(fixed_cost+unit_price*var_amount,GRB.MINIMIZE) 
m.update()

In [19]:
#Constrains
sumlist = []
for i in range(len(num_stores)):
    sumlist.append(sum(binary[i]))
    m.addConstr(sumlist[i],GRB.EQUAL, 1)
    m.addConstr(demand[i],GRB.GREATER_EQUAL, storeinfo[i][1])
    m.addConstr(demand[i],GRB.LESS_EQUAL, 12000)
m.update()
m.optimize()

Optimize a model with 3300 rows, 12100 columns and 13200 nonzeros
Model has 11000 quadratic objective terms
Variable types: 0 continuous, 12100 integer (11000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 2e+02]
  QObjective range [2e+02, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 6.244963e+07
Presolve removed 3300 rows and 12100 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3.00448e+07 6.24496e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.004475777521e+07, best bound 3.004475777521e+07, gap 0.0000%


In [28]:
cur.execute('delete from results')
for i in range(len(m.getVars())):
    if m.getVars()[i].x == 1.0:
        cur.execute(cur.execute("insert into results(dc_id,store_id) values (%s,%s)",(re.findall(r'\d+', str(m.getVars()[i]))[0],re.findall(r'\d+', str(m.getVars()[i]))[1])))
        db.commit()